In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/raw/wine-quality.csv")
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
df.describe().T[["min", "max"]]

,min,max
fixed_acidity,3.80000,14.20000
volatile_acidity,0.08000,1.10000
citric_acid,0.00000,1.66000
residual_sugar,0.60000,65.80000
chlorides,0.00900,0.34600
free_sulfur_dioxide,2.00000,289.00000
total_sulfur_dioxide,9.00000,440.00000
density,0.98711,1.03898
pH,2.72000,3.82000
sulphates,0.22000,1.08000


In [6]:
df.describe().loc[["min", "max"]].to_json("schema_in.json")

In [7]:
class NotInRange(Exception):
    def __init__(self, message="value not in range") -> None:
        # self.input = input
        self.message = message
        super().__init__(self.message)

In [8]:
raise NotInRange

NotInRange: value not in range

In [12]:
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import yaml
import os

def read_params(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config['mlflow_config']

    remote_server_uri = mlflow_config['remote_server_uri']
    model_name = mlflow_config['registered_model_name']

    mlflow.set_registry_uri(remote_server_uri)
    runs = mlflow.search_runs(experiment_ids=['1'])

    print(runs)

    lowest_mae_val = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"]==lowest_mae_val]["run_id"][0]
    
    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        if mv["run_id"] == lowest_run_id:
            current_version = mv['version']
            logged_model = mv['source']
            pprint(mv, index=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Production",
            )
        else:
            current_version = mv['version']
            pprint(mv, index=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Staging",
            )

log_production_model(config_path="../params.yaml")

Empty DataFrame
Columns: [run_id, experiment_id, status, artifact_uri, start_time, end_time]
Index: []


KeyError: 'metrics.mae'